# Análisis de Transacciones Fraudulentas con ANN

- Marco Antonio Jurado carnet 20308
#### Fecha: 24/5/2024

### Resumen
Este proyecto tiene como objetivo la detección de transacciones fraudulentas utilizando el modelo de Machine Learning ANN. El análisis incluye:

- Entrenamiento del modelo
- Evaluación del modelo
- Visualización de resultados

### Contenido
1. Entrenamiento del Modelo
2. Evaluación del Modelo
3. Conclusiones y Trabajo Futuro

In [1]:
#!pip install lightgbm
#!pip install --upgrade lightgbm
#!pip install category_encoders
#!pip install geopy

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, auc, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import os
import shutil

Abrimos los datasets de cada mes que ya hemos balanceado

In [3]:
input_dir = 'final_datasets'

En este caso vamos a cargar los set de datos individuales de cada mes que hemos generado en previos pasos

In [4]:
# Diccionario para almacenar los datasets
monthly_datasets = {}

# Listar todos los archivos en el directorio
for file_name in os.listdir(input_dir):
    if file_name.startswith('resampled_data_') and file_name.endswith('.csv'):
        # Construir el path completo del archivo
        file_path = os.path.join(input_dir, file_name)

        # Extraer la fecha del nombre del archivo
        date_str = file_name[len('resampled_data_'):-4]
        month = pd.to_datetime(date_str, format='%Y_%m')

        # Cargar el dataset
        monthly_datasets[month] = pd.read_csv(file_path)
        print(f"Dataset para {month.strftime('%Y-%m')} cargado desde: {file_path}")

Dataset para 2019-01 cargado desde: final_datasets\resampled_data_2019_01.csv
Dataset para 2019-02 cargado desde: final_datasets\resampled_data_2019_02.csv
Dataset para 2019-03 cargado desde: final_datasets\resampled_data_2019_03.csv
Dataset para 2019-04 cargado desde: final_datasets\resampled_data_2019_04.csv
Dataset para 2019-05 cargado desde: final_datasets\resampled_data_2019_05.csv
Dataset para 2019-06 cargado desde: final_datasets\resampled_data_2019_06.csv
Dataset para 2019-07 cargado desde: final_datasets\resampled_data_2019_07.csv
Dataset para 2019-08 cargado desde: final_datasets\resampled_data_2019_08.csv
Dataset para 2019-09 cargado desde: final_datasets\resampled_data_2019_09.csv
Dataset para 2019-10 cargado desde: final_datasets\resampled_data_2019_10.csv
Dataset para 2019-11 cargado desde: final_datasets\resampled_data_2019_11.csv
Dataset para 2019-12 cargado desde: final_datasets\resampled_data_2019_12.csv
Dataset para 2020-01 cargado desde: final_datasets\resampled_dat

### Entrenamiento no incremental

Vamos a configurar el modelo

In [5]:
# Configuración del modelo ANN
def create_ann_model(input_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

Vamos a combinar la data en un solo set para hacer el entrenamiento no incremental. Aqui tenemos que tener nuestras variables X y Y respectivamente para poder hacer el entrenamiento.

In [6]:
combined_data = pd.concat(monthly_datasets.values())
X = combined_data.drop(['is_fraud'], axis=1)
y = combined_data['is_fraud']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

Carpetas necesarias para los outputs

In [7]:
folder_path = 'ANN_combined_performance'
roc_folder = os.path.join(folder_path, 'roc_curves')
conf_matrix_folder = os.path.join(folder_path, 'conf_matrices')
metrics_folder = os.path.join(folder_path, 'metrics_texts')

for path in [folder_path, roc_folder, conf_matrix_folder, metrics_folder]:
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)

Entrenemos...

In [8]:
# Escalar los datos
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Crear y entrenar el modelo ANN
model = create_ann_model(X_train.shape[1])
model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

# Calcular métricas
y_pred = model.predict(X_val)
y_pred_binary = (y_pred >= 0.5).astype(int)
f1 = f1_score(y_val, y_pred_binary)
accuracy = accuracy_score(y_val, y_pred_binary)
precision = precision_score(y_val, y_pred_binary)
recall = recall_score(y_val, y_pred_binary)
cm = confusion_matrix(y_val, y_pred_binary)

Epoch 1/10
26807/26807 [==============================] - 47s 2ms/step - loss: 0.0540 - accuracy: 0.9809
Epoch 2/10
26807/26807 [==============================] - 41s 2ms/step - loss: 0.0334 - accuracy: 0.9881
Epoch 3/10
26807/26807 [==============================] - 39s 1ms/step - loss: 0.0267 - accuracy: 0.9906
Epoch 4/10
26807/26807 [==============================] - 39s 1ms/step - loss: 0.0233 - accuracy: 0.9918
Epoch 5/10
26807/26807 [==============================] - 39s 1ms/step - loss: 0.0212 - accuracy: 0.9925
Epoch 6/10
26807/26807 [==============================] - 41s 2ms/step - loss: 0.0198 - accuracy: 0.9930
Epoch 7/10
26807/26807 [==============================] - 39s 1ms/step - loss: 0.0186 - accuracy: 0.9934
Epoch 8/10
26807/26807 [==============================] - 41s 2ms/step - loss: 0.0176 - accuracy: 0.9939
Epoch 9/10
26807/26807 [==============================] - 40s 1ms/step - loss: 0.0170 - accuracy: 0.9941
Epoch 10/10
22977/22977 [==============================

Generamos los resultados del entrenamiento. Empezamos con la curva ROC

In [9]:
fpr, tpr, _ = roc_curve(y_val, y_pred)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(10, 5))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Combined Data')
plt.legend(loc="lower right")
plt.savefig(os.path.join(roc_folder, 'roc_curve_combined.png'))
plt.close()

Matriz de confusión

In [10]:
plt.figure(figsize=(6, 5))
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix for Combined Data')
plt.savefig(os.path.join(conf_matrix_folder, 'conf_matrix_combined.png'))
plt.close()

<Figure size 600x500 with 0 Axes>

Metricas F1, Precisión, Efectividad y Recall

In [11]:
metrics_file_path = os.path.join(metrics_folder, 'model_metrics.txt')
with open(metrics_file_path, 'w') as metrics_file:
    metrics_file.write("Metrics for Combined Data:\n")
    metrics_file.write(f"F1 Score: {f1:.2f}\n")
    metrics_file.write(f"Accuracy: {accuracy:.2f}\n")
    metrics_file.write(f"Precision: {precision:.2f}\n")
    metrics_file.write(f"Recall: {recall:.2f}\n")
    metrics_file.write(f"Confusion Matrix:\n{cm}\n\n")

print("Revisa las carpetas dentro de 'ANN_combined_performance' para ver el desempeño del modelo.")

Revisa las carpetas dentro de 'ANN_combined_performance' para ver el desempeño del modelo.


### Entrenamiento incremental
Vamos a configurar el modelo inicial

In [12]:
# Configuración del modelo ANN
def create_ann_model(input_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

Carpetas necesarias para tener los outputs generados del entrenamiento y validación

In [13]:
# Configurar directorios
folder_path = 'ANN_incremental_performance'
roc_folder = os.path.join(folder_path, 'roc_curves')
conf_matrix_folder = os.path.join(folder_path, 'conf_matrices')
metrics_folder = os.path.join(folder_path, 'metrics_texts')

# Asegurarse de que las carpetas estén limpias
for path in [folder_path, roc_folder, conf_matrix_folder, metrics_folder]:
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)

In [14]:
metrics_file_path = os.path.join(metrics_folder, 'model_metrics.txt')
with open(metrics_file_path, 'w') as metrics_file:
    month_count = 0  # Contador para el reentrenamiento

    # Iterar sobre cada segmento mensual
    for name, month_data in monthly_datasets.items():
        print(f"Entrenando con datos de: {name}")

        # Dividir los datos en entrenamiento y validación
        X = month_data.drop(['is_fraud'], axis=1)
        y = month_data['is_fraud']
        X_train_month, X_val_month, y_train_month, y_val_month = train_test_split(
            X, y, test_size=0.3, random_state=42)

        # Escalar los datos
        scaler = StandardScaler()
        X_train_month = scaler.fit_transform(X_train_month)
        X_val_month = scaler.transform(X_val_month)

        # Reestablecer el modelo cada 3 meses
        if month_count % 3 == 0:
            model = create_ann_model(X_train_month.shape[1])

        # Entrenamiento con validación
        model.fit(X_train_month, y_train_month, epochs=5, batch_size=64, verbose=1)

        # Incrementar el contador de meses
        month_count += 1

        # Calcular métricas
        y_pred = model.predict(X_val_month)
        y_pred_binary = (y_pred >= 0.5).astype(int)
        f1 = f1_score(y_val_month, y_pred_binary)
        accuracy = accuracy_score(y_val_month, y_pred_binary)
        precision = precision_score(y_val_month, y_pred_binary)
        recall = recall_score(y_val_month, y_pred_binary)
        cm = confusion_matrix(y_val_month, y_pred_binary)

        # Graficar ROC
        fpr, tpr, _ = roc_curve(y_val_month, y_pred)
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=(10, 5))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'ROC Curve for {name.strftime("%B %Y")}')
        plt.legend(loc="lower right")
        plt.savefig(os.path.join(roc_folder, f'roc_curve_{name.strftime("%Y_%m")}.png'))
        plt.close()

        # Graficar matriz de confusión
        plt.figure(figsize=(6, 5))
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot(cmap=plt.cm.Blues)
        plt.title(f'Confusion Matrix for {name.strftime("%B %Y")}')
        plt.savefig(os.path.join(conf_matrix_folder, f'conf_matrix_{name.strftime("%Y_%m")}.png'))
        plt.close()

        # Escribir métricas en el archivo
        metrics_file.write(f"Metrics for {name.strftime('%B %Y')}:\n")
        metrics_file.write(f"F1 Score: {f1:.2f}\n")
        metrics_file.write(f"Accuracy: {accuracy:.2f}\n")
        metrics_file.write(f"Precision: {precision:.2f}\n")
        metrics_file.write(f"Recall: {recall:.2f}\n")
        metrics_file.write(f"Confusion Matrix:\n{cm}\n\n")

print("Revisa las carpetas dentro de 'ANN_incremental_performance' para ver el desempeño de los modelos.")

Entrenando con datos de: 2019-01-01 00:00:00
Epoch 1/5
757/757 [==============================] - 2s 2ms/step - loss: 0.1144 - accuracy: 0.9607
Epoch 2/5
757/757 [==============================] - 1s 1ms/step - loss: 0.0353 - accuracy: 0.9897
Epoch 3/5
757/757 [==============================] - 1s 1ms/step - loss: 0.0200 - accuracy: 0.9941
Epoch 4/5
757/757 [==============================] - 1s 1ms/step - loss: 0.0136 - accuracy: 0.9962
Epoch 5/5
649/649 [==============================] - 1s 1ms/step
Entrenando con datos de: 2019-02-01 00:00:00
Epoch 1/5
718/718 [==============================] - 1s 2ms/step - loss: 0.0301 - accuracy: 0.9911
Epoch 2/5
718/718 [==============================] - 1s 1ms/step - loss: 0.0102 - accuracy: 0.9971
Epoch 3/5
718/718 [==============================] - 1s 1ms/step - loss: 0.0071 - accuracy: 0.9982
Epoch 4/5
718/718 [==============================] - 1s 1ms/step - loss: 0.0057 - accuracy: 0.9985
Epoch 5/5
616/616 [==============================] - 

C:\Users\marco\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_plot\confusion_matrix.py:136: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots()


Entrenando con datos de: 2020-09-01 00:00:00
Epoch 1/5
1007/1007 [==============================] - 2s 2ms/step - loss: 0.0163 - accuracy: 0.9951
Epoch 2/5
1007/1007 [==============================] - 2s 2ms/step - loss: 0.0045 - accuracy: 0.9986
Epoch 3/5
1007/1007 [==============================] - 2s 2ms/step - loss: 0.0027 - accuracy: 0.9992
Epoch 4/5
1007/1007 [==============================] - 2s 2ms/step - loss: 0.0021 - accuracy: 0.9994
Epoch 5/5
863/863 [==============================] - 1s 984us/step


C:\Users\marco\AppData\Local\Temp\ipykernel_40116\1990006100.py:42: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(10, 5))
C:\Users\marco\AppData\Local\Temp\ipykernel_40116\1990006100.py:55: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(6, 5))


Entrenando con datos de: 2020-10-01 00:00:00
Epoch 1/5
1004/1004 [==============================] - 2s 2ms/step - loss: 0.0679 - accuracy: 0.9791
Epoch 2/5
1004/1004 [==============================] - 2s 2ms/step - loss: 0.0163 - accuracy: 0.9951
Epoch 3/5
1004/1004 [==============================] - 2s 2ms/step - loss: 0.0088 - accuracy: 0.9973
Epoch 4/5
1004/1004 [==============================] - 2s 2ms/step - loss: 0.0054 - accuracy: 0.9986
Epoch 5/5
860/860 [==============================] - 1s 1ms/step
Entrenando con datos de: 2020-11-01 00:00:00
Epoch 1/5
1053/1053 [==============================] - 2s 2ms/step - loss: 0.0225 - accuracy: 0.9942
Epoch 2/5
1053/1053 [==============================] - 2s 2ms/step - loss: 0.0059 - accuracy: 0.9984
Epoch 3/5
1053/1053 [==============================] - 1s 1ms/step - loss: 0.0035 - accuracy: 0.9992
Epoch 4/5
1053/1053 [==============================] - 2s 1ms/step - loss: 0.0025 - accuracy: 0.9994
Epoch 5/5
902/902 [==================

...

<Figure size 600x500 with 0 Axes>

<Figure size 600x500 with 0 Axes>

<Figure size 600x500 with 0 Axes>

<Figure size 600x500 with 0 Axes>

<Figure size 600x500 with 0 Axes>

En este caso hemos notado una diferencia en el tiempo de ejecución siendo de casi 8 minutos para el secuencial normal y de casi 5 minutos para el incremental. Sin embargo la efectividad del modelo no se ve afectada por lo que podemos decir entonces que en este caso es mejor el uso de entrenamiento incremental aún cuando querramos agregar nuevos datos de un nuevo mes.